# Hyperparameter Tuning
## Imports

In [1]:
import yfinance as yf
from sklearn import svm
import warnings

warnings.filterwarnings("ignore")

In [2]:
from RiskLabAI.optimization.hyper_parameter_tuning import *

## Data

In [3]:
btc = yf.Ticker('BTC-USD').history(start='2021-02-01', end='2021-04-01')['Close']
btc_df = pd.DataFrame(btc)
btc_df['Expected 55'] = btc.ewm(span=55, adjust=False).mean()
btc_df['Expected 15'] = btc.ewm(span=15, adjust=False).mean()
btc_df['Target'] = -btc.diff(-1).apply(np.sign).replace(0, -1)
btc_df = btc_df.dropna()

btc_df.index = btc_df.index.tz_convert(None)
btc_df = btc_df.tz_localize(None)
btc= btc_df['Close']

In [4]:
target_positions = btc_df['Target']
data = btc_df[['Close', 'Expected 55', 'Expected 15']]
times = pd.Series(btc.index.to_series().shift(-1).ffill(), index=btc.index)

## Setting up the tuning parameters

In [5]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()

## Running the hyperparameter tuning
In this step, using Purged K-Fold as CV, we will run the hyperparameter tuning on the data and print the best parameters.

In [6]:
data = pd.DataFrame(data, index=times.index)
target_positions = pd.Series(target_positions, index=times.index)
print(clf_hyper_fit(data, target_positions, times, svc, parameters))

SVC(C=1, kernel='linear')
